In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# prepare the data
data = datasets.load_breast_cancer()
x , y = data.data, data.target

n_samples, n_features = x.shape

 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

#scale
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_test = torch.from_numpy(x_test.astype(np.float32))
x_train = torch.from_numpy(x_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [4]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [5]:
model = LogisticRegression(n_features)

#loss and optimizer
criterion = nn.BCELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.Adam(params=model.parameters() , lr=0.01)


In [6]:
epochs = 1000
for epoch in range(epochs):
    #forward and loss
    y_pred = model(x_train)
    loss = criterion(y_pred , y_train)

    #backward pass
    loss.backward()

    #updates
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1) % 100 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item()}')
with torch.no_grad():
    y_pred = model(x_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f' accuracy = {acc}')

epoch: 100, loss = 0.08626608550548553
epoch: 200, loss = 0.06098167598247528
epoch: 300, loss = 0.05015765130519867
epoch: 400, loss = 0.04386495053768158
epoch: 500, loss = 0.03965698555111885
epoch: 600, loss = 0.03660406172275543
epoch: 700, loss = 0.0342676155269146
epoch: 800, loss = 0.032409992069005966
epoch: 900, loss = 0.03088853508234024
epoch: 1000, loss = 0.02961086668074131
 accuracy = 0.9561403393745422


tensor(109)